In [1]:
import numpy as np 
import torch
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import warnings
from tensorflow.keras.applications import VGG19
warnings.filterwarnings('ignore')
#import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow.keras import Sequential 
from tensorflow.keras.utils import Sequence, to_categorical, plot_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Input, MaxPooling2D, concatenate, BatchNormalization, Activation, Dropout
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from random import sample, choice
from PIL import Image
import cv2
import warnings
warnings.filterwarnings("ignore")
import albumentations as albu
import time
import os
from tqdm.notebook import tqdm
from sklearn.preprocessing import OneHotEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2022-09-20 07:55:08.122989: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-20 07:55:08.123829: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 07:55:08.126035: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 07:55:08.127365: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [3]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [4]:
from __future__ import print_function
from sklearn.metrics import confusion_matrix
import numpy as np
import warnings
from tensorflow.keras import regularizers
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Model

import warnings
warnings.filterwarnings("ignore")
from keras import applications
from keras.applications import *
from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, Adamax

In [5]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]
input_shape = IMAGE_SIZE + [3]

train_path = '../input/trainset/TRAINING DATA'
valid_path = '../input/validation-dataset/VALIDATION DATA'
test_path  = '../input/testdataset/TESTING DATA'

In [6]:
# Import the ResNet50 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
import tensorflow as tf
#pretrained_model = tensorflow.keras.applications.resnet50.ResNet50(input_shape=input_shape, include_top=False, weights= 'imagenet' )
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False

2022-09-20 07:55:20.766598: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 07:55:20.767879: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 07:55:20.769004: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 07:55:20.770362: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 07:55:20.771388: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [7]:
# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False

In [8]:
folders = glob('../input/trainset/TRAINING DATA/*')

In [9]:
print (folders)

['../input/trainset/TRAINING DATA/OUTSIDE-Train', '../input/trainset/TRAINING DATA/DRINK-Train', '../input/trainset/TRAINING DATA/FOOD-Train', '../input/trainset/TRAINING DATA/MENU-Train', '../input/trainset/TRAINING DATA/INSIDE-Train']


In [10]:
# our layers - you can add more if you want
x = Flatten()(resnet.output)

In [11]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [12]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [13]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [14]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)
validate_datagen = ImageDataGenerator(rescale = 1./255)

In [15]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('../input/trainset/TRAINING DATA',
                                                 target_size = (224, 224),
                                                 batch_size = 128,
                                                 class_mode = 'categorical')

Found 149979 images belonging to 5 classes.


In [16]:
test_set = test_datagen.flow_from_directory('../input/testdataset/TESTING DATA',
                                            target_size = (224, 224),
                                            batch_size = 128,
                                            class_mode = 'categorical')

Found 18747 images belonging to 5 classes.


In [17]:
validate_set = validate_datagen.flow_from_directory('../input/validation-dataset/VALIDATION DATA',
                                            target_size = (224, 224),
                                            batch_size = 128,
                                            class_mode = 'categorical')

Found 18750 images belonging to 5 classes.


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=validate_set,
  epochs=10
)

2022-09-20 07:56:55.976192: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-09-20 07:57:01.005669: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1172/1172 [==============================] - 2633s 2s/step - loss: 0.8269 - accuracy: 0.7451 - val_loss: 0.6294 - val_accuracy: 0.7926
Epoch 2/10
1172/1172 [==============================] - 2660s 2s/step - loss: 0.6694 - accuracy: 0.7777 - val_loss: 0.6232 - val_accuracy: 0.7951
Epoch 3/10
1172/1172 [==============================] - 2640s 2s/step - loss: 0.6655 - accuracy: 0.7819 - val_loss: 0.5684 - val_accuracy: 0.8097
Epoch 4/10
 373/1172 [========>.....................] - ETA: 27:28 - loss: 0.5868 - accuracy: 0.8011

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
model.save("./output")

In [ ]:
'''
import os
from PIL import Image
from tqdm import tqdm
folder_path = '../input/training-data/TRAINING DATA'
extensions = ['jpg']
for fldr in os.listdir(folder_path):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in tqdm(os.listdir(sub_folder_path)):
        file_path = os.path.join(sub_folder_path, filee)
        #print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        try:
            im = Image.open(file_path)
            rgb_im = im.convert('RGB')
        except Exception as e:
            print("Exception : " , e )
            os.remove(file_path)
        #if filee.split('.')[1] not in extensions:
            #extensions.append(filee.split('.')[1])
'''

In [ ]:
print(extensions)